In [2]:
!pip3 install torch transformers datasets

Defaulting to user installation because normal site-packages is not writeable


In [3]:
from transformers import T5ForConditionalGeneration, T5Tokenizer, AdamW
from torch.utils.data import DataLoader, Dataset
from datasets import load_dataset
from torch.nn.utils.rnn import pad_sequence

/Users/jahnavisuresh/Library/Python/3.9/lib/python/site-packages/urllib3/__init__.py:35: NotOpenSSLWarning: urllib3 v2 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(
/Users/jahnavisuresh/Library/Python/3.9/lib/python/site-packages/bitsandbytes/cextension.py:34: UserWarning: The installed version of bitsandbytes was compiled without GPU support. 8-bit optimizers, 8-bit multiplication, and GPU quantization are unavailable.
  warn("The installed version of bitsandbytes was compiled without GPU support. "


'NoneType' object has no attribute 'cadam32bit_grad_fp32'


In [4]:
dataset = load_dataset("spider")

Generating train split:   0%|          | 0/7000 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/1034 [00:00<?, ? examples/s]

In [5]:
model_name = "t5-small"
tokenizer = T5Tokenizer.from_pretrained(model_name)
model = T5ForConditionalGeneration.from_pretrained(model_name)

tokenizer_config.json:   0%|          | 0.00/2.32k [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.39M [00:00<?, ?B/s]

You are using the default legacy behaviour of the <class 'transformers.models.t5.tokenization_t5.T5Tokenizer'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thoroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


config.json:   0%|          | 0.00/1.21k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/242M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/147 [00:00<?, ?B/s]

In [6]:
class SQLDataset(Dataset):
    def __init__(self, examples):
        self.examples = examples

    def __len__(self):
        return len(self.examples)

    def __getitem__(self, idx):
        source_text = f"generate SQL command: {self.examples['question'][idx]}"
        target_text = self.examples['query'][idx]

        source_tokens = tokenizer(source_text, truncation=True, return_tensors="pt")['input_ids'][0]
        target_tokens = tokenizer(target_text, truncation=True, return_tensors="pt")['input_ids'][0]

        return {"input_ids": source_tokens, "labels": target_tokens}

def collate_fn(batch):
    inputs = pad_sequence([item["input_ids"] for item in batch], batch_first=True, padding_value=tokenizer.pad_token_id)
    labels = pad_sequence([item["labels"] for item in batch], batch_first=True, padding_value=tokenizer.pad_token_id)
    max_length = max(inputs.size(1), labels.size(1))
    inputs = torch.nn.functional.pad(inputs, (0, max_length - inputs.size(1)))
    labels = torch.nn.functional.pad(labels, (0, max_length - labels.size(1)))
    return {"input_ids": inputs, "labels": labels}

In [7]:
train_dataset = SQLDataset(dataset["train"])
validation_dataset = SQLDataset(dataset["validation"])

In [8]:
train_dataloader = DataLoader(train_dataset, batch_size=4, shuffle=True, collate_fn=collate_fn)

In [48]:
num_epochs = 20
learning_rate = 2e-5
optimizer = AdamW(model.parameters(), lr=learning_rate)

In [49]:
import torch
for epoch in range(num_epochs):
    model.train()
    total_loss = 0
    for batch in train_dataloader:
        inputs = batch["input_ids"]
        labels = batch["labels"]
        outputs = model(inputs, labels=labels)
        loss = outputs.loss
        total_loss += loss.item()

        # Backward pass and optimization
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

    average_loss = total_loss / len(train_dataloader)
    print(f"Epoch {epoch + 1}/{num_epochs}, Average Loss: {average_loss}")

Epoch 1/20, Average Loss: 0.6862480348859514
Epoch 2/20, Average Loss: 0.6147596886754036
Epoch 3/20, Average Loss: 0.5597205617427826
Epoch 4/20, Average Loss: 0.5167337958131518
Epoch 5/20, Average Loss: 0.4801048256414277
Epoch 6/20, Average Loss: 0.45454782849550246
Epoch 7/20, Average Loss: 0.42383134664808003
Epoch 8/20, Average Loss: 0.40484655922225543
Epoch 9/20, Average Loss: 0.38503022661805153
Epoch 10/20, Average Loss: 0.36383310897009713
Epoch 11/20, Average Loss: 0.3512051869588239
Epoch 12/20, Average Loss: 0.3372640271782875
Epoch 13/20, Average Loss: 0.32277861030612676
Epoch 14/20, Average Loss: 0.3111431175087179
Epoch 15/20, Average Loss: 0.29837157737995895
Epoch 16/20, Average Loss: 0.2889571975554739
Epoch 17/20, Average Loss: 0.27941734197310036
Epoch 18/20, Average Loss: 0.2678367601994957
Epoch 19/20, Average Loss: 0.26103652819139617
Epoch 20/20, Average Loss: 0.25089518458076887


In [50]:
model.save_pretrained("fine_tuned_sql_model")

In [56]:
input_text = "generate SQL command: What is the average salary of employees?"
input_ids = tokenizer(input_text, return_tensors="pt")["input_ids"]
output = model.generate(input_ids,max_length=75)
generated_sql = tokenizer.decode(output[0], skip_special_tokens=True)

print("Input Text:", input_text)
print("Generated SQL:", generated_sql)

Input Text: generate SQL command: What is the average salary of employees?
Generated SQL: SELECT avg(Salary) FROM Employees
